In [3]:
import os

import seaborn as sns

import numpy as np
import xarray as xr
import dask
from dask.distributed import Client, SSHCluster
from dask.diagnostics import Profiler

In [7]:

# cluster = SSHCluster(
#     ["localhost", "localhost", "localhost"],#"panoseti-dfs1", "panoseti-dfs2"],
#     connect_options={"known_hosts": None},
#     n_workers=2,
#     worker_options={"nthreads": 2},
#     scheduler_options={"port": 0, "dashboard_address": ":8797"}
# )

client = Client()#cluster)

# Print the dashboard link to observe the computation
print(f"Dask dashboard link: {client.dashboard_link}")


/Users/nico/anaconda3/envs/zarr-py313/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58710 instead
  warnings.warn(


Dask dashboard link: http://127.0.0.1:58710/status


In [10]:
!pwd

/Users/nico/panoseti/panoseti_zarr


In [13]:
# zarr_in_path = zarr_out_path
zarr_in_path = "zarr_test/pano_img.zarr"
assert os.path.exists(zarr_in_path)


ds = xr.open_datatree(
    zarr_in_path,
    consolidated=False,
    engine='zarr',
    chunks='auto',
    cache=True
)
ds.images

<xarray.DataArray 'images' (time: 4128, y: 32, x: 32)> Size: 8MB
dask.array<open_dataset-images, shape=(4128, 32, 32), dtype=uint16, chunksize=(4128, 32, 32), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x
Attributes:
    _ARRAY_DIMENSIONS:  ['time', 'y', 'x']
    pff_metadata:       {'bpp': '2', 'debug': 'TRUNCATED', 'dp': 'img16', 'mo...
    source_pff_file:    start_2024-07-25T04_34_46Z.dp_img16.bpp_2.module_1.se...

In [14]:
out_path = 'img_medsub.zarr'

if 'img' in zarr_in_path:
    out_path = 'img_medsub.zarr'
    median_img = ds.images.median('time')
    median_subtraction = ds.images - median_img
    store_operation = median_subtraction.to_zarr(
        out_path,
        mode='w',
        consolidated=False,
        compute=False,
        zarr_format=3
    )
    store_operation.compute()

In [15]:
ds_clean = xr.open_datatree(
    out_path,
    consolidated=False,
    engine='zarr',
    chunks={},
    # cache=True
)

ds_clean.images

<xarray.DataArray 'images' (time: 4128, y: 32, x: 32)> Size: 34MB
dask.array<open_dataset-images, shape=(4128, 32, 32), dtype=float64, chunksize=(4128, 32, 32), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x